## Model Trainer Trial-runs 

In [ ]:
import os 

In [ ]:
chdir()

## Entity Trial-runs 

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int 
    per_device_train_batch_size: int 
    weight_decay: float
    logging_steps: int 
    evaluation_strategy: str 
    eval_steps: int 
    save_steps: float
    gradient_accumulation_steps: int


## ConfigurationManager Trial-runs 

In [ ]:
#importing all project paths and modules necessary for project configurations 
from textSummerizer.constants import  *
from textSummerizer.utils.common import read_yaml, create_directories

In [ ]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer 
        params=self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path, 
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps, 
            per_device_train_batch_size=params.per_device_train_batch_size, 
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps, 
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_step =params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config

## Componets Trial-runs 

In [ ]:
#Importing dependencies 
from transfomers import TrainingArguments, Trainer
from transfomers import DataCollatorForSeq25eq
from transfomers import AutoModelForSeq25eqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch 

In [ ]:
class ModelTrainer: 
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def train(self):
        device= "cuba" if tourch.cuba.is_vailable() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq25eqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq25eq(tokenizer, model=model_pegasus)

        #loading data 
        data_samsum_pt =load_from_disk(self.config.data_path)

        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps, 
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps, 
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_step =self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus, 
            args=trainer_args, 
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator, 
            train_dataset=data_samsum_pt["test"], 
            eval_dataset=data_samsum_pt["validation"]
        )
        
        trainer.train()

        #save model 
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

## Pipeline Trial-runs 

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config =config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e